# Análisis de Tasas de Falsos Positivos y Negativos

Este notebook demuestra el uso de la función `analisis_falsos_positivos_negativos` para evaluar la calidad de diferentes esquemas de calificación frente a un "Oráculo".

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import notas as nu

pd.set_option('display.float_format', '{:.2f}'.format)
np.random.seed(42)

In [2]:
# 1. Generar Datos
# import config as cf
# import config_astrobio as cf
import config_meccel as cf
config = nu.autoconfigura_items(cf.config_evaluacion)
df = nu.genera_datos(config, N=1000)
df.head()

Autoconfigurando items correlacionados...
 > Proyecto2: Ajustado vs Proyecto1 (x1.0)


,Quices,Taller,Proyecto1,Proyecto2,Proyecto3,nota_concepto
0,3.54,3.33,0.98,3.24,2.87,4.50
1,3.98,1.64,1.24,3.78,4.45,4.00
2,1.16,4.40,0.38,2.54,2.73,2.50
3,2.43,3.68,4.68,2.31,3.34,2.50
4,1.88,4.31,0.49,3.57,1.66,3.50


In [3]:
nu.muestra_pesos(config)


PESOS FINALES DE EVALUACIÓN
ITEM                      PESO      
-----------------------------------
Quices                    0.1818
Taller                    0.2727
Proyecto1                 0.1509
Proyecto2                 0.1509
Proyecto3                 0.1527
Nota_Concepto             0.0909
-----------------------------------
TOTAL SUMA                1.0000


In [4]:
config

{'items_normales': [{'nombre': 'Quices',
   'debilidad': 3.5,
   'peso_min': 0.1,
   'peso_max': 0.3,
   'peso_sugerido': 0.2,
   'tipo': 'avanzado',
   'umbral': 3.0,
   'peso_final': np.float64(0.18181818181818182)},
  {'nombre': 'Taller',
   'debilidad': 3.5,
   'peso_min': 0.1,
   'peso_max': 0.3,
   'peso_sugerido': 0.3,
   'tipo': 'facil',
   'umbral': 3.0,
   'peso_final': np.float64(0.2727272727272727)},
  {'nombre': 'Proyecto1',
   'debilidad': 2.5,
   'peso_min': 0.05,
   'peso_max': 0.2,
   'peso_sugerido': 0.166,
   'tipo': 'clave',
   'umbral': 3.0,
   'peso_final': np.float64(0.1509090909090909)},
  {'nombre': 'Proyecto2',
   'debilidad': 2.5,
   'correlacionado_con': 'Proyecto1',
   'factor_correlacion': 1.0,
   'tipo': 'avanzado',
   'umbral': 3.0,
   'peso_min': 0.05,
   'peso_max': 0.2,
   'peso_sugerido': 0.166,
   'peso_final': np.float64(0.1509090909090909)}],
 'item_definitorio': {'nombre': 'Proyecto3',
  'muerte_subita': 2.5,
  'peso_maximo': 0.3,
  'tipo': 'exam

In [5]:
nu.muestra_pesos(config)


PESOS FINALES DE EVALUACIÓN
ITEM                      PESO      
-----------------------------------
Quices                    0.1818
Taller                    0.2727
Proyecto1                 0.1509
Proyecto2                 0.1509
Proyecto3                 0.1527
Nota_Concepto             0.0909
-----------------------------------
TOTAL SUMA                1.0000


In [6]:
# 2. Calcular Notas con Diferentes Métodos

# Método Avanzado
res_adv = nu.calcula_promedio_con_umbrales_avanzado(df, config)

# Promedio Clásico (ya viene en res_adv)
df['Promedio_Clasico'] = res_adv['Promedio_Clasico']

# Método Simple
df['Nota_Final_Simple'] = nu.calcula_promedio_con_umbrales_simple(df, config)
df['Nota_Final_Avanzada'] = res_adv['Nota_Final']

df.head()

,Quices,Taller,Proyecto1,Proyecto2,Proyecto3,nota_concepto,Promedio_Clasico,Nota_Final_Simple,Nota_Final_Avanzada
0,3.54,3.33,0.98,3.24,2.87,4.50,3.04,1.09,1.72
1,3.98,1.64,1.24,3.78,4.45,4.00,3.00,2.38,1.72
2,1.16,4.40,0.38,2.54,2.73,2.50,2.50,2.00,2.50
3,2.43,3.68,4.68,2.31,3.34,2.50,3.24,2.15,3.00
4,1.88,4.31,0.49,3.57,1.66,3.50,2.70,2.16,2.70


## 3. Análisis de Métricas

Evaluamos cada método contra el Oráculo.

In [7]:
methods = ['Promedio_Clasico', 'Nota_Final_Simple', 'Nota_Final_Avanzada']

results = {}

for method in methods:
    print(f"\n{'='*20} {method} {'='*20}")
    res = nu.analisis_falsos_positivos_negativos(df, method, config)
    results[method] = res
    
    print(f"FPR (Falsos Aprobados): {res['FPR']:.2%}")
    print(f"FNR (Falsos Reprobados): {res['FNR']:.2%}")
    print(f"Accuracy: {res['Accuracy']:.2%}")
    print(f"TP: {res['TP']} | TN: {res['TN']} | FP: {res['FP']} | FN: {res['FN']}")


==================== Promedio_Clasico ====================
FPR (Falsos Aprobados): 52.47%
FNR (Falsos Reprobados): 14.57%
Accuracy: 64.70%
TP: 387 | TN: 260 | FP: 287 | FN: 66

==================== Nota_Final_Simple ====================
FPR (Falsos Aprobados): 3.66%
FNR (Falsos Reprobados): 73.51%
Accuracy: 64.70%
TP: 120 | TN: 527 | FP: 20 | FN: 333

==================== Nota_Final_Avanzada ====================
FPR (Falsos Aprobados): 22.85%
FNR (Falsos Reprobados): 59.60%
Accuracy: 60.50%
TP: 183 | TN: 422 | FP: 125 | FN: 270


## 4. Ejemplos de Errores

Visualizamos casos donde los métodos fallan.

In [8]:
# Ejemplo: Falsos Positivos del Promedio Clásico
fp_classic = results['Promedio_Clasico']['FP_Examples']
if not fp_classic.empty:
    print("\nEjemplos Falsos Positivos - Promedio Clásico:")
    display(fp_classic.head())
else:
    print("No hay Falsos Positivos en Promedio Clásico")


Ejemplos Falsos Positivos - Promedio Clásico:


,Quices,Taller,Proyecto1,Proyecto2,Proyecto3,nota_concepto,Promedio_Clasico,Nota_Final_Simple,Nota_Final_Avanzada,Decision_Oraculo
3,2.43,3.68,4.68,2.31,3.34,2.50,3.24,2.15,3.00,0
6,2.01,3.73,3.73,2.66,2.44,2.50,3.00,2.36,3.00,0
7,2.09,4.42,4.37,0.70,5.00,2.50,3.34,0.63,1.20,0
15,2.18,4.03,1.56,3.15,2.52,3.50,3.00,2.33,3.00,0
24,3.57,3.81,1.39,3.84,1.63,4.50,3.14,1.06,1.98,0


In [9]:
# Ejemplo: Falsos Negativos del Método Simple
fn_simple = results['Nota_Final_Simple']['FN_Examples']
if not fn_simple.empty:
    print("\nEjemplos Falsos Negativos - Método Simple:")
    display(fn_simple.head())
else:
    print("No hay Falsos Negativos en Método Simple")


Ejemplos Falsos Negativos - Método Simple:


,Quices,Taller,Proyecto1,Proyecto2,Proyecto3,nota_concepto,Promedio_Clasico,Nota_Final_Simple,Nota_Final_Avanzada,Decision_Oraculo
0,3.54,3.33,0.98,3.24,2.87,4.50,3.04,1.09,1.72,1
1,3.98,1.64,1.24,3.78,4.45,4.00,3.00,2.38,1.72,1
5,3.94,4.22,1.47,3.14,3.37,4.50,3.49,1.82,2.52,1
13,2.35,3.93,3.91,3.44,3.34,3.50,3.44,2.78,3.23,1
14,3.24,3.62,0.90,3.54,3.43,4.50,3.18,1.05,1.65,1
